# Export Functionality Demo

This notebook demonstrates the export functionality of the RAG Engine Mini.

## Learning Objectives

By the end of this notebook, you will understand:
1. How the export functionality works in the RAG Engine
2. The different export formats supported
3. How to use the export API endpoints
4. The architecture of the export service

In [ ]:
import sys
import os
from pathlib import Path
import asyncio

# Add the project root to the path
project_root = Path("../")
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print("Environment set up successfully")

## Understanding the Export Architecture

The export functionality follows the same architectural patterns as the rest of the RAG Engine:

1. **Port/Adapter Pattern**: The `ExportServicePort` defines the interface
2. **Dependency Injection**: Services are injected through the container
3. **Separation of Concerns**: Export logic is separate from API logic
4. **Format Support**: Multiple export formats (PDF, Markdown, CSV, JSON)

In [ ]:
# Let's look at the export service definition
from src.application.services.export_service import ExportService, ExportFormat

print("Available export formats:")
for fmt in ExportFormat:
    print(f"- {fmt.value}")

print(f"\nExport service class: {ExportService.__name__}")
print(f"Methods implemented: {[method for method in dir(ExportService) if not method.startswith('_')]}\n")

## Using the Export Service

Let's see how to use the export service to generate different export formats:

In [ ]:
# Import required classes
from src.application.services.export_service import ExportService, ExportFormat
from src.domain.entities import TenantId
from src.adapters.persistence.placeholder import PlaceholderDocumentRepo

# Create a placeholder document repository for demonstration
doc_repo = PlaceholderDocumentRepo()

# Create the export service
export_service = ExportService(
    document_repo=doc_repo
)

print("Export service initialized successfully")
print(f"Document repository: {type(doc_repo).__name__}")

## Testing Different Export Formats

Let's test the export functionality with different formats:

In [ ]:
# Test tenant ID
tenant_id = TenantId("demo-tenant")

# Test content for different formats
test_content = "# Sample Document\n\nThis is a sample document for export testing.\n\n- Item 1\n- Item 2\n- Item 3"
test_data = [{"name": "John", "age": 30, "city": "New York"}, {"name": "Jane", "age": 25, "city": "Boston"}]

print("Testing export to different formats:\n")

# Export to JSON
try:
    json_result = asyncio.run(export_service.export_to_json(
        tenant_id=tenant_id,
        data=test_data,
        filename="test.json"
    ))
    print("✅ JSON export successful")
    print(f"JSON preview: {json_result[:100]}...")
except Exception as e:
    print(f"❌ JSON export failed: {e}")

# Export to CSV
try:
    csv_result = asyncio.run(export_service.export_to_csv(
        tenant_id=tenant_id,
        rows=test_data,
        filename="test.csv"
    ))
    print(f"\n✅ CSV export successful")
    print(f"CSV preview: {csv_result[:100]}...")
except Exception as e:
    print(f"❌ CSV export failed: {e}")

# Export to Markdown
try:
    md_result = asyncio.run(export_service.export_to_markdown(
        tenant_id=tenant_id,
        content=test_content,
        filename="test.md"
    ))
    print(f"\n✅ Markdown export successful")
    print(f"Markdown preview: {md_result[:100]}...")
except Exception as e:
    print(f"❌ Markdown export failed: {e}")

# Export to PDF (just checking if the method exists)
try:
    pdf_result = asyncio.run(export_service.export_to_pdf(
        tenant_id=tenant_id,
        content=test_content,
        filename="test.pdf"
    ))
    print(f"\n✅ PDF export successful")
    print(f"PDF size: {len(pdf_result)} bytes")
except Exception as e:
    print(f"❌ PDF export failed: {e}")

## Creating an Export Job

Now let's test creating an actual export job, which would typically run in the background:

In [ ]:
# Create an export job
try:
    export_job = asyncio.run(export_service.create_export_job(
        tenant_id=tenant_id,
        format=ExportFormat.JSON,
        content_type="documents",
        filters=None
    ))
    
    print("Export job created successfully:")
    print(f"- Job ID: {export_job.job_id}")
    print(f"- Format: {export_job.format}")
    print(f"- Content type: {export_job.content_type}")
    print(f"- Status: {export_job.status}")
    print(f"- Created at: {export_job.created_at}")
    
except Exception as e:
    print(f"Failed to create export job: {e}")

## Preview Export Functionality

Let's test the preview functionality, which allows users to see a sample of what would be exported:

In [ ]:
# Test the preview functionality
try:
    preview = asyncio.run(export_service.preview_export(
        tenant_id=tenant_id,
        format=ExportFormat.MARKDOWN,
        content_type="documents",
        filters=None,
        limit=1
    ))
    
    print("Export preview generated successfully:")
    print(f"Preview content: {preview[:200]}...")
    
except Exception as e:
    print(f"Failed to generate preview: {e}")

## API Endpoints

The export functionality is also available through API endpoints. Let's examine the routes:

In [ ]:
# Import the API router to see available endpoints
from src.api.v1.routes_exports import router

print("Export API routes:")
for route in router.routes:
    if hasattr(route, 'methods') and hasattr(route, 'path'):
        print(f"- {route.methods}: {route.path}")

print(f"\nTotal export API routes: {len([r for r in router.routes if hasattr(r, 'methods')])}")

## Integration with Use Cases

The export functionality is integrated with the application through use cases:

In [ ]:
from src.application.use_cases.export_use_case import ExportUseCase, ExportRequest

# Create the use case
export_use_case = ExportUseCase(export_service=export_service)

print(f"Export use case: {ExportUseCase.__name__}")
print(f"Available methods: {[method for method in dir(ExportUseCase) if not method.startswith('_')]}")

# Test creating an export request
request = ExportRequest(
    format=ExportFormat.JSON,
    content_type="documents",
    filters=None
)

print(f"\nExport request created:")
print(f"- Format: {request.format}")
print(f"- Content type: {request.content_type}")

## Summary

In this notebook, we explored the export functionality of the RAG Engine Mini:

1. **Architecture**: The export service follows the same architectural patterns as the rest of the system
2. **Formats**: Supports PDF, Markdown, CSV, and JSON export formats
3. **API**: Provides comprehensive API endpoints for creating and managing exports
4. **Integration**: Seamlessly integrates with the use case and dependency injection patterns
5. **Flexibility**: Allows for both immediate exports and background jobs

This export functionality completes the feature set that was marked as pending in the project completion checklist, bringing the RAG Engine Mini closer to full completion!